In [31]:
import os, json
import pandas as pd
from tqdm import tqdm

import boto3
import pyarrow.parquet as pymysql
from dotenv import load_dotenv
from io import BytesIO
import io
from PIL import Image

import gc
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [32]:
load_dotenv('/home/flexmatch_ftp/ftp/multimodal_categorizer/config/.env')

aws_access_key = os.getenv("aws_accessKey")
aws_secret_key = os.getenv("aws_secretKey")
region_name='ap-northeast-2'

# S3 클라이언트 생성
s3_client = boto3.client('s3', aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_key, region_name=region_name)

In [33]:
final_img_path_df = pd.read_parquet("/home/flexmatch_ftp/ftp/multimodal_categorizer/category_classification/final_img_path_df.parquet")

In [35]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
# from PIL import Image
# import requests
# import warnings

# # 경고 메시지 무시 (예: bitsandbytes 관련 경고)
# warnings.filterwarnings("ignore")

# # 1. 모델 및 토크나이저 로드
# # Qwen2.5-VL-7B-Instruct 모델을 로드합니다.
# # VRAM 절약을 위해 load_in_4bit=True 옵션을 사용하여 4비트 양자화로 로드
# # device_map="auto"를 통해 사용 가능한 GPU에 자동으로 모델을 분배
# print("모델 및 토크나이저 로딩 중...")
# model_id = "Qwen/Qwen2.5-VL-7B-Instruct"
# tokenizer = AutoProcessor.from_pretrained(model_id)
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     model_id,
#     torch_dtype=torch.bfloat16, # bfloat16은 더 넓은 동적 범위를 가지며, 요즘 GPU에서 잘 지원됩니다.
#     device_map="auto",          # 사용 가능한 GPU에 자동으로 로드
#     trust_remote_code=True      
# )
# model.eval() # 모델을 평가 모드로 설정

# print("모델 로딩 완료.")

# # 2. 이미지 준비

# bucket_name = 'flexmatch-data'
# obj = s3_client.get_object(Bucket=bucket_name, Key=final_img_path_df.loc[2, "s3_path"])
# image = Image.open(BytesIO(obj['Body'].read())).convert("RGB")


# # 3. 프롬프트 구성 및 이미지 캡셔닝
# # Qwen2.5-VL은 특정 프롬프트 형식을 기대합니다.
# # <image> 토큰은 이미지의 위치를 나타냅니다.
# # "What is in this image?"와 같은 질문을 통해 이미지 캡션을 요청합니다.
# print("\n이미지 캡셔닝 시작...")

# # 하나의 이미지에 대한 캡션 요청
# query_single = "What is in this image?"
# messages_single = [
#     {"role": "user",
#       "content": [
#         {"type": "image", "content": image},
#         {"type": "text", "content": query_single}
#     ]}
# ]
# text_single = tokenizer.apply_chat_template(messages_single, tokenize=False, add_generation_prompt=True)
# input_ids_single = tokenizer(text_single, return_tensors="pt").input_ids.to(model.device)

# with torch.no_grad():
#     output_single = model.generate(input_ids_single,
#                                    max_new_tokens=200, # 생성할 최대 토큰 수
#                                    do_sample=False,    # 샘플링 비활성화 (결정론적 출력)
#                                    pad_token_id=tokenizer.eos_token_id)

# response_single = tokenizer.decode(output_single[0][input_ids_single.shape[1]:], skip_special_tokens=True)
# print(f"\n[단일 이미지 캡션]:\n{response_single}")

# # 더 자세한 질문으로 이미지 설명 요청
# query_detailed = "Describe this image in detail."
# messages_detailed = [
#     {"role": "user", "content": [
#         {"type": "image", "content": image},
#         {"type": "text", "content": query_detailed}
#     ]}
# ]
# text_detailed = tokenizer.apply_chat_template(messages_detailed, tokenize=False, add_generation_prompt=True)
# input_ids_detailed = tokenizer(text_detailed, return_tensors="pt").input_ids.to(model.device)

# with torch.no_grad():
#     output_detailed = model.generate(input_ids_detailed,
#                                      max_new_tokens=300,
#                                      do_sample=True,      # 샘플링 활성화 (좀 더 다양한 출력)
#                                      temperature=0.7,     # 샘플링 온도 조절
#                                      top_p=0.9,           # top-p 샘플링
#                                      pad_token_id=tokenizer.eos_token_id)

# response_detailed = tokenizer.decode(output_detailed[0][input_ids_detailed.shape[1]:], skip_special_tokens=True)
# print(f"\n[상세 이미지 설명]:\n{response_detailed}")

# print("\n작업 완료.")

In [44]:
import torch
import json
import gc
import os
import warnings
from transformers import AutoTokenizer, AutoProcessor
from transformers import Qwen2_5_VLForConditionalGeneration
from PIL import Image
from io import BytesIO
from tqdm import tqdm
import boto3
import traceback

warnings.filterwarnings("ignore")

# 1. 모델 로드
print("모델 및 토크나이저 로딩 중..")
model_id = "Qwen/Qwen2.5-VL-7B-Instruct"
tokenizer = AutoProcessor.from_pretrained(model_id)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)
model.eval()
print("모델 로딩 완료.")

def load_image_from_s3(bucket, key):
    """S3에서 이미지 로드"""
    try:
        obj = s3_client.get_object(Bucket=bucket, Key=key)
        img_bytes = obj["Body"].read()
        img = Image.open(BytesIO(img_bytes)).convert("RGB")
        return img
    except Exception as e:
        print(f"S3 로드 오류 ({key}): {str(e)}")
        traceback.print_exc()
        return None

def caption_image(image, model, tokenizer, query="What is in this image?"):
    """단일 이미지 캡셔닝"""
    try:
        print(f"  [DEBUG] 이미지 타입: {type(image)}, 크기: {image.size if hasattr(image, 'size') else 'N/A'}")
        
        # Qwen2.5-VL 형식: text에 <image> 토큰 포함
        messages = [
            {"role": "user", "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": query}
            ]}
        ]
        print(f"  [DEBUG] 메시지 구성 완료")
        
        # 이미지와 텍스트를 함께 처리
        # text_prompt는 "<image> " + query 형식
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            return_tensors="pt",
            return_dict=True
        ).to(model.device)
        print(f"  [DEBUG] Tokenizer 완료, input keys: {inputs.keys()}")
        
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=200,
                do_sample=False,
            )
        print(f"  [DEBUG] 모델 생성 완료, output shape: {output.shape}")
        
        # 생성된 토큰만 추출 (입력 길이 제외)
        generated_ids = output[0][inputs['input_ids'].shape[1]:]
        caption = tokenizer.decode(generated_ids, skip_special_tokens=True)
        print(f"  [DEBUG] 디코딩 완료: {caption[:50]}")
        
        return caption
    except Exception as e:
        print(f"❌ 캡셔닝 오류: {str(e)}")
        traceback.print_exc()
        return None
    finally:
        torch.cuda.empty_cache()

# 2. 먼저 단일 이미지로 테스트
print("\n" + "="*50)
print("단일 이미지 테스트")
print("="*50)

bucket_name = 'flexmatch-data'
test_keys = final_img_path_df['s3_path'].to_list()[:1]

for test_key in test_keys:
    print(f"\n테스트 키: {test_key}")
    test_img = load_image_from_s3(bucket_name, test_key)
    
    if test_img is not None:
        print("✓ 이미지 로드 성공")
        caption = caption_image(test_img, model, tokenizer)
        print(f"✓ 최종 캡션: {caption}")
    else:
        print("✗ 이미지 로드 실패")

# 3. 배치 처리 (이제 테스트 후 진행)
print("\n" + "="*50)
print("배치 처리 시작")
print("="*50)

keys = final_img_path_df['s3_path'].to_list()[:3]
batch_size = 100
output_file = "qwen_captions_results.jsonl"

processed_keys = set()
if os.path.exists(output_file):
    with open(output_file, 'r') as f:
        for line in f:
            data = json.loads(line)
            processed_keys.add(data['key'])
    print(f"이전에 처리한 파일: {len(processed_keys)}개")

print(f"\n총 {len(keys)}개 이미지 캡셔닝 시작...")
with open(output_file, 'a') as f:
    for i, key in enumerate(tqdm(keys, desc="이미지 캡셔닝")):
        if key in processed_keys:
            continue
        
        img = load_image_from_s3(bucket_name, key)
        if img is not None:
            caption = caption_image(img, model, tokenizer)
        else:
            caption = None
        
        result = {"key": key, "caption": caption}
        f.write(json.dumps(result, ensure_ascii=False) + '\n')
        f.flush()
        
        del img
        gc.collect()
        
        if (i + 1) % batch_size == 0:
            torch.cuda.empty_cache()
            print(f"배치 {(i+1)//batch_size} 완료")

print("\n캡셔닝 완료!")

# 4. 결과 확인
print("\n=== 처리 결과 ===")
with open(output_file, 'r') as f:
    lines = f.readlines()
    for line in lines[-5:]:
        data = json.loads(line)
        caption = data['caption']
        print(f"{data['key']}: {caption}")

del model
torch.cuda.empty_cache()

모델 및 토크나이저 로딩 중..


Loading weights: 100%|██████████| 729/729 [00:02<00:00, 341.53it/s, Materializing param=model.visual.patch_embed.proj.weight]                          


모델 로딩 완료.

단일 이미지 테스트

테스트 키: instagram-data/tables/INSTAGRAM_POST_IMAGE_SAMPLE/media_image/17841400038455668/18088296511639307.jpg
✓ 이미지 로드 성공
  [DEBUG] 이미지 타입: <class 'PIL.Image.Image'>, 크기: (2160, 1215)
  [DEBUG] 메시지 구성 완료
  [DEBUG] Tokenizer 완료, input keys: KeysView({'input_ids': tensor([[151644,   8948,    198,  ...,     30, 151645,    198]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'), 'pixel_values': tensor([[1.4194, 1.4340, 1.4340,  ..., 1.6055, 1.6055, 1.6198],
        [1.4340, 1.4340, 1.4486,  ..., 1.6624, 1.6624, 1.6766],
        [1.4340, 1.4340, 1.4486,  ..., 1.6198, 1.6198, 1.6482],
        ...,
        [1.3902, 1.3756, 1.3610,  ..., 1.5913, 1.6055, 1.6198],
        [1.3610, 1.3610, 1.3610,  ..., 1.5771, 1.5913, 1.5913],
        [1.3756, 1.3756, 1.3610,  ..., 1.5771, 1.6055, 1.6198]],
       device='cuda:0'), 'image_grid_thw': tensor([[  1,  86, 154]], device='cuda:0')})
  [DEBUG] 모델 생성 완료, output shape: torch.Size([1, 342

이미지 캡셔닝:   0%|          | 0/3 [00:00<?, ?it/s]

  [DEBUG] 이미지 타입: <class 'PIL.Image.Image'>, 크기: (2160, 1215)
  [DEBUG] 메시지 구성 완료
  [DEBUG] Tokenizer 완료, input keys: KeysView({'input_ids': tensor([[151644,   8948,    198,  ...,     30, 151645,    198]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'), 'pixel_values': tensor([[1.4194, 1.4340, 1.4340,  ..., 1.6055, 1.6055, 1.6198],
        [1.4340, 1.4340, 1.4486,  ..., 1.6624, 1.6624, 1.6766],
        [1.4340, 1.4340, 1.4486,  ..., 1.6198, 1.6198, 1.6482],
        ...,
        [1.3902, 1.3756, 1.3610,  ..., 1.5913, 1.6055, 1.6198],
        [1.3610, 1.3610, 1.3610,  ..., 1.5771, 1.5913, 1.5913],
        [1.3756, 1.3756, 1.3610,  ..., 1.5771, 1.6055, 1.6198]],
       device='cuda:0'), 'image_grid_thw': tensor([[  1,  86, 154]], device='cuda:0')})
  [DEBUG] 모델 생성 완료, output shape: torch.Size([1, 3421])
  [DEBUG] 디코딩 완료:  addCriterion
The image shows two circular objects


...

  [DEBUG] 이미지 타입: <class 'PIL.Image.Image'>, 크기: (1080, 1440)
  [DEBUG] 메시지 구성 완료
  [DEBUG] Tokenizer 완료, input keys: KeysView({'input_ids': tensor([[151644,   8948,    198,  ...,     30, 151645,    198]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'), 'pixel_values': tensor([[-1.7923, -1.7923, -1.7923,  ..., -1.4802, -1.4802, -1.4802],
        [-1.7923, -1.7923, -1.7923,  ..., -1.4802, -1.4802, -1.4802],
        [-1.7923, -1.7923, -1.7923,  ..., -1.4802, -1.4802, -1.4802],
        ...,
        [-1.7923, -1.7923, -1.7923,  ..., -1.4802, -1.4802, -1.3522],
        [-1.7923, -1.7923, -1.7923,  ..., -1.4802, -1.4802, -1.4802],
        [-1.7923, -1.7923, -1.7923,  ..., -1.4802, -1.4802, -1.3522]],
       device='cuda:0'), 'image_grid_thw': tensor([[  1, 102,  78]], device='cuda:0')})
  [DEBUG] 모델 생성 완료, output shape: torch.Size([1, 2109])
  [DEBUG] 디코딩 완료:  addCriterion
The image shows a white Tesla Model 


이미지 캡셔닝:  67%|██████▋   | 2/3 [00:06<00:03,  3.25s/it]

  [DEBUG] 이미지 타입: <class 'PIL.Image.Image'>, 크기: (855, 569)
  [DEBUG] 메시지 구성 완료
  [DEBUG] Tokenizer 완료, input keys: KeysView({'input_ids': tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198, 151652, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 15

이미지 캡셔닝: 100%|██████████| 3/3 [00:09<00:00,  3.11s/it]


캡셔닝 완료!

=== 처리 결과 ===
instagram-data/tables/INSTAGRAM_POST_IMAGE_SAMPLE/media_image/17841400038455668/18088296511639307.jpg:  addCriterion
The image shows two circular objects that appear to be parts of a coffee grinder, specifically the burr assembly and the base plate. The object on the left is the burr assembly, which contains the grinding mechanism. The object on the right is the base plate, which holds the burr assembly and provides a stable surface for the grinder. Both components have the brand name "Archee" written on them.
instagram-data/tables/INSTAGRAM_POST_IMAGE_SAMPLE/media_image/17841400038455668/17888713530296800.jpg:  addCriterion
The image shows a white Tesla Model Y parked in an indoor parking garage. The car has a license plate that reads "229 4715". Below the image, there is Korean text that translates to "Tesla Model Y Zuniper Scoco Protection Film PPF Self DIY." This suggests that the image might be related to a do-it-yourself (DIY) project involving applying pr

In [ ]:
import torch
import json
import gc
import os
import warnings
from transformers import AutoTokenizer, AutoProcessor
from transformers import Qwen2_5_VLForConditionalGeneration
from PIL import Image
from io import BytesIO
from tqdm import tqdm
import boto3
import traceback

warnings.filterwarnings("ignore")


print("모델 및 토크나이저 로딩 중")
model_id = "Qwen/Qwen2.5-VL-7B-Instruct"
tokenizer = AutoProcessor.from_pretrained(model_id)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

model.eval()
print("모델 로딩 완료.")

def load_image_from_s3(bucket, key):
    """S3에서 이미지 로드"""
    try:
        obj = s3_client.get_object(Bucket=bucket, Key=key)
        img_bytes = obj["Body"].read()
        img = Image.open(BytesIO(img_bytes)).convert("RGB")
        return img
    except Exception as e:
        print(f"S3 로드 오류 ({key}): {str(e)}")
        traceback.print_exc()
        return None

def caption_image(image, model, tokenizer, query="Describe this image briefly in 1-2 sentences. Then, describe the emotional tone or mood of this image using adjectives like: warm, cold, energetic, calm, melancholic, joyful, mysterious, etc."):
    """단일 이미지 캡셔닝"""
    try:
        print(f"  [DEBUG] 이미지 타입: {type(image)}, 크기: {image.size if hasattr(image, 'size') else 'N/A'}")
        
        # Qwen2.5-VL 형식: text에 <image> 토큰 포함
        messages = [
            {"role": "user", "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": query}
            ]}
        ]
        print(f"  [DEBUG] 메시지 구성 완료")
        
        # 이미지와 텍스트를 함께 처리
        # text_prompt는 "<image> " + query 형식
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            return_tensors="pt",
            return_dict=True
        ).to(model.device)
        print(f"  [DEBUG] Tokenizer 완료, input keys: {inputs.keys()}")
        
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=200,
                do_sample=False,
            )
        print(f"  [DEBUG] 모델 생성 완료, output shape: {output.shape}")
        
        # 생성된 토큰만 추출 (입력 길이 제외)
        generated_ids = output[0][inputs['input_ids'].shape[1]:]
        caption = tokenizer.decode(generated_ids, skip_special_tokens=True)
        print(f"  [DEBUG] 디코딩 완료: {caption[:50]}")
        
        return caption
    except Exception as e:
        print(f"❌ 캡셔닝 오류: {str(e)}")
        traceback.print_exc()
        return None
    finally:
        torch.cuda.empty_cache()

# 2. 먼저 단일 이미지로 테스트
print("\n" + "="*50)
print("단일 이미지 테스트")
print("="*50)

bucket_name = 'flexmatch-data'
test_keys = final_img_path_df['s3_path'].to_list()[:1]

for test_key in test_keys:
    print(f"\n테스트 키: {test_key}")
    test_img = load_image_from_s3(bucket_name, test_key)
    
    if test_img is not None:
        print("✓ 이미지 로드 성공")
        caption = caption_image(test_img, model, tokenizer)
        print(f"✓ 최종 캡션: {caption}")
    else:
        print("✗ 이미지 로드 실패")

# 3. 배치 처리 (이제 테스트 후 진행)
print("\n" + "="*50)
print("배치 처리 시작")
print("="*50)

keys = final_img_path_df['s3_path'].to_list()[:3]
batch_size = 1  # 하나씩 처리 후 메모리 정리
output_file = "qwen_captions_results.jsonl"

processed_keys = set()
if os.path.exists(output_file):
    with open(output_file, 'r') as f:
        for line in f:
            data = json.loads(line)
            processed_keys.add(data['key'])
    print(f"이전에 처리한 파일: {len(processed_keys)}개")

print(f"\n총 {len(keys)}개 이미지 캡셔닝 시작...")
with open(output_file, 'a') as f:
    for i, key in enumerate(tqdm(keys, desc="이미지 캡셔닝")):
        # 이미 처리한 것은 스킵
        if key in processed_keys:
            print(f"  [SKIP] {key} - 이미 처리됨")
            continue
        
        print(f"\n[{i+1}/{len(keys)}] 처리 중: {key}")
        img = load_image_from_s3(bucket_name, key)
        caption = None
        
        if img is not None:
            caption = caption_image(img, model, tokenizer)
            print(f"  [SUCCESS] 캡션 생성 완료")
        else:
            print(f"  [FAILED] 이미지 로드 실패")
        
        result = {"key": key, "caption": caption}
        f.write(json.dumps(result, ensure_ascii=False) + '\n')
        f.flush()
        print(f"  [SAVED] 결과 저장 완료: {caption[:50] if caption else 'None'}")
        
        if img is not None:
            del img
        gc.collect()
        
        if (i + 1) % batch_size == 0:
            torch.cuda.empty_cache()
            print(f"배치 {(i+1)//batch_size} 완료, GPU 메모리 정리됨")

print("\n캡셔닝 완료!")


del model
torch.cuda.empty_cache()

모델 및 토크나이저 로딩 중


Loading weights: 100%|██████████| 729/729 [00:02<00:00, 348.86it/s, Materializing param=model.visual.patch_embed.proj.weight]                          


모델 로딩 완료.

단일 이미지 테스트

테스트 키: instagram-data/tables/INSTAGRAM_POST_IMAGE_SAMPLE/media_image/17841400038455668/18088296511639307.jpg
✓ 이미지 로드 성공
  [DEBUG] 이미지 타입: <class 'PIL.Image.Image'>, 크기: (2160, 1215)
  [DEBUG] 메시지 구성 완료
  [DEBUG] Tokenizer 완료, input keys: KeysView({'input_ids': tensor([[151644,   8948,    198,  ...,     13, 151645,    198]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'), 'pixel_values': tensor([[1.4194, 1.4340, 1.4340,  ..., 1.6055, 1.6055, 1.6198],
        [1.4340, 1.4340, 1.4486,  ..., 1.6624, 1.6624, 1.6766],
        [1.4340, 1.4340, 1.4486,  ..., 1.6198, 1.6198, 1.6482],
        ...,
        [1.3902, 1.3756, 1.3610,  ..., 1.5913, 1.6055, 1.6198],
        [1.3610, 1.3610, 1.3610,  ..., 1.5771, 1.5913, 1.5913],
        [1.3756, 1.3756, 1.3610,  ..., 1.5771, 1.6055, 1.6198]],
       device='cuda:0'), 'image_grid_thw': tensor([[  1,  86, 154]], device='cuda:0')})
  [DEBUG] 모델 생성 완료, output shape: torch.Size([1, 341

이미지 캡셔닝:   0%|          | 0/3 [00:00<?, ?it/s]


[1/3] 처리 중: instagram-data/tables/INSTAGRAM_POST_IMAGE_SAMPLE/media_image/17841400038455668/18088296511639307.jpg
  [DEBUG] 이미지 타입: <class 'PIL.Image.Image'>, 크기: (2160, 1215)
  [DEBUG] 메시지 구성 완료
  [DEBUG] Tokenizer 완료, input keys: KeysView({'input_ids': tensor([[151644,   8948,    198,  ...,     13, 151645,    198]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'), 'pixel_values': tensor([[1.4194, 1.4340, 1.4340,  ..., 1.6055, 1.6055, 1.6198],
        [1.4340, 1.4340, 1.4486,  ..., 1.6624, 1.6624, 1.6766],
        [1.4340, 1.4340, 1.4486,  ..., 1.6198, 1.6198, 1.6482],
        ...,
        [1.3902, 1.3756, 1.3610,  ..., 1.5913, 1.6055, 1.6198],
        [1.3610, 1.3610, 1.3610,  ..., 1.5771, 1.5913, 1.5913],
        [1.3756, 1.3756, 1.3610,  ..., 1.5771, 1.6055, 1.6198]],
       device='cuda:0'), 'image_grid_thw': tensor([[  1,  86, 154]], device='cuda:0')})
  [DEBUG] 모델 생성 완료, output shape: torch.Size([1, 3415])
  [DEBUG] 디코딩 완료:  addCri

...

이미지 캡셔닝:  67%|██████▋   | 2/3 [00:04<00:02,  2.25s/it]

배치 2 완료, GPU 메모리 정리됨

[3/3] 처리 중: instagram-data/tables/INSTAGRAM_POST_IMAGE_SAMPLE/media_image/17841400038455668/17971563212887318.jpg
  [DEBUG] 이미지 타입: <class 'PIL.Image.Image'>, 크기: (855, 569)
  [DEBUG] 메시지 구성 완료
  [DEBUG] Tokenizer 완료, input keys: KeysView({'input_ids': tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198, 151652, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 1516

이미지 캡셔닝:  67%|██████▋   | 2/3 [00:07<00:03,  3.71s/it]

  [DEBUG] 모델 생성 완료, output shape: torch.Size([1, 779])
  [DEBUG] 디코딩 완료:  addCriterion

 addCriterion

 addCriterion

 addC
  [SUCCESS] 캡션 생성 완료


UnicodeEncodeError: 'ascii' codec can't encode characters in position 313-316: ordinal not in range(128)

In [54]:
warnings.filterwarnings("ignore")


# 1. 모델 로드 (양자화 없음)
print("모델 및 토크나이저 로딩 중..")
model_id = "Qwen/Qwen2.5-VL-7B-Instruct"
tokenizer = AutoProcessor.from_pretrained(model_id)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)
model.eval()
print("모델 로딩 완료.")


모델 및 토크나이저 로딩 중..


Loading weights: 100%|██████████| 729/729 [00:02<00:00, 349.60it/s, Materializing param=model.visual.patch_embed.proj.weight]                          


모델 로딩 완료.


In [ ]:
def load_image_from_s3(bucket, key):
    """S3에서 이미지 로드"""
    try:
        obj = s3_client.get_object(Bucket=bucket, Key=key)
        img_bytes = obj["Body"].read()
        img = Image.open(BytesIO(img_bytes)).convert("RGB")
        return img
    except Exception as e:
        print(f"S3 로드 오류 ({key}): {str(e)}")
        return None

def clean_caption(caption):
    """캡션에서 불필요한 토큰 및 아티팩트 제거"""
    if caption is None:
        return None
    
    # 내부 토큰 제거
    caption = caption.replace("addCriterion", "").strip()
    caption = caption.replace("自动生成", "").strip()
    
    # 연속 개행 정리
    while "\n\n\n" in caption:
        caption = caption.replace("\n\n\n", "\n\n")
    
    # 앞뒤 공백/개행 제거
    caption = caption.strip()
    
    return caption if caption else None

def caption_image(image, model, tokenizer):
    """이미지에서 설명과 감성 추출"""
    try:
        # 프롬프트: 간단한 설명 + 감성 분석
        query = """Describe this image briefly in 1-2 sentences. 
Then, describe the emotional tone or mood of this image using adjectives like: warm, cold, energetic, calm, melancholic, joyful, mysterious, etc."""
        
        messages = [
            {"role": "user", "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": query}
            ]}
        ]
        
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            return_tensors="pt",
            return_dict=True
        ).to(model.device)
        
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=150,
                do_sample=False,
            )
        
        # 생성된 토큰만 추출
        generated_ids = output[0][inputs['input_ids'].shape[1]:]
        caption = tokenizer.decode(generated_ids, skip_special_tokens=True)
        
        return caption
    except Exception as e:
        print(f"캡셔닝 오류: {str(e)}")
        traceback.print_exc()
        return None
    finally:
        torch.cuda.empty_cache()

# 2. 배치 처리
bucket_name = 'flexmatch-data'
keys = final_img_path_df['s3_path'].to_list()[21103:]  # 전체 데이터
batch_size = 100 
output_file = "qwen_emotion_captions_2.jsonl"

# 기존 결과 확인 (중단됐을 경우 재개)
processed_keys = set()
if os.path.exists(output_file):
    with open(output_file, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            processed_keys.add(data['key'])
    print(f"이전에 처리한 파일: {len(processed_keys)}개 (재개 모드)")
else:
    print("새로 시작합니다")

# 배치 처리
print(f"\n총 {len(keys)}개 이미지 캡셔닝 시작...")
print(f"배치 크기: {batch_size}, 예상 배치 수: {len(keys)//batch_size + 1}")

with open(output_file, 'a', encoding='utf-8') as f:
    for i, key in enumerate(tqdm(keys, desc="이미지 캡셔닝")):
        # 이미 처리한 것은 스킵
        if key in processed_keys:
            continue
        
        # 이미지 로드
        img = load_image_from_s3(bucket_name, key)
        caption = None
        
        if img is not None:
            caption = caption_image(img, model, tokenizer)
            caption = clean_caption(caption)  # 아티팩트 제거
        
        # 결과 저장
        result = {"key": key, "caption": caption}
        f.write(json.dumps(result, ensure_ascii=False) + '\n')
        f.flush()
        
        # 메모리 정리
        if img is not None:
            del img
        gc.collect()
        
        # 배치마다 GPU 캐시 정리
        if (i + 1) % batch_size == 0:
            torch.cuda.empty_cache()
            completed = len(processed_keys) + (i + 1)
            percentage = (completed / len(keys)) * 100
            print(f"\n배치 {(i+1)//batch_size} 완료 ({completed}/{len(keys)}, {percentage:.1f}%), GPU 메모리 정리됨")

print("\n" + "="*50)
print("캡셔닝 완료!")
print("="*50)

# 3. 최종 통계
total_processed = len(processed_keys) + len(keys)
with open(output_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    total_lines = len(lines)
    success_count = sum(1 for line in lines if json.loads(line).get('caption') is not None)

print(f"\n최종 통계:")
print(f"  총 이미지: {len(keys)}개")
print(f"  처리 완료: {total_lines}개")
print(f"  성공: {success_count}개 ({success_count/total_lines*100:.1f}%)")
print(f"  실패: {total_lines - success_count}개")

# 4. 샘플 결과
print("\n=== 샘플 결과 (마지막 3개) ===")
try:
    with open(output_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines[-3:]:
            data = json.loads(line)
            if data['caption']:
                print(f"\n키: {data['key']}")
                print(f"캡션:\n{data['caption']}")
            else:
                print(f"\n키: {data['key']}")
                print(f"캡션: [실패]")
except Exception as e:
    print(f"샘플 읽기 오류: {e}")

# 메모리 정리
del model
torch.cuda.empty_cache()

새로 시작합니다

총 122563개 이미지 캡셔닝 시작...
배치 크기: 100, 예상 배치 수: 1226


이미지 캡셔닝:   0%|          | 100/122563 [04:57<88:15:12,  2.59s/it]


배치 1 완료 (100/122563, 0.1%), GPU 메모리 정리됨


이미지 캡셔닝:   0%|          | 200/122563 [09:37<74:30:12,  2.19s/it] 


배치 2 완료 (200/122563, 0.2%), GPU 메모리 정리됨


...


배치 210 완료 (21000/122563, 17.1%), GPU 메모리 정리됨


이미지 캡셔닝:  17%|█▋        | 21100/122563 [17:00:10<65:49:08,  2.34s/it] 


배치 211 완료 (21100/122563, 17.2%), GPU 메모리 정리됨


이미지 캡셔닝:  17%|█▋        | 21104/122563 [17:00:23<81:45:37,  2.90s/it]


KeyboardInterrupt: 

In [ ]:
# Jupyter에서 로컬에 모델 저장
from huggingface_hub import snapshot_download
snapshot_download(
    "Qwen/Qwen2.5-VL-7B-Instruct",
    local_dir="/home/flexmatch_ftp/ftp/multimodal_categorizer/models/"
)

In [56]:
from vllm import LLM, SamplingParams

warnings.filterwarnings("ignore")

print("모델 및 토크나이저 로딩 중..")
model_id = "Qwen/Qwen2.5-VL-7B-Instruct"
tokenizer = AutoProcessor.from_pretrained(model_id)
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     model_id,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
#     trust_remote_code=True
# )

model = LLM("Qwen/Qwen2.5-VL-7B-Instruct", dtype="bfloat16", gpu_memory_utilization=0.9)
model.eval()
print("모델 로딩 완료.")


ImportError: /home/flexmatch_ftp/miniconda3/envs/blip2/lib/python3.10/site-packages/vllm/_C.abi3.so: undefined symbol: _ZN3c106ivalue14ConstantString6createENSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE

In [ ]:
def load_image_from_s3(bucket, key):
    """S3에서 이미지 로드"""
    try:
        obj = s3_client.get_object(Bucket=bucket, Key=key)
        img_bytes = obj["Body"].read()
        img = Image.open(BytesIO(img_bytes)).convert("RGB")
        return img
    except Exception as e:
        print(f"S3 로드 오류 ({key}): {str(e)}")
        return None

def clean_caption(caption):
    """캡션에서 불필요한 토큰 및 아티팩트 제거"""
    if caption is None:
        return None
    
    # 내부 토큰 제거
    caption = caption.replace("addCriterion", "").strip()
    caption = caption.replace("自动生成", "").strip()
    
    # 연속 개행 정리
    while "\n\n\n" in caption:
        caption = caption.replace("\n\n\n", "\n\n")
    
    # 앞뒤 공백/개행 제거
    caption = caption.strip()
    
    return caption if caption else None

def caption_image(image, model, tokenizer):
    """이미지에서 설명과 감성 추출"""
    try:
        # 프롬프트: 간단한 설명 + 감성 분석
        query = """Describe this image briefly in 1-2 sentences. 
Then, describe the emotional tone or mood of this image using adjectives like: warm, cold, energetic, calm, melancholic, joyful, mysterious, etc."""
        
        messages = [
            {"role": "user", "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": query}
            ]}
        ]
        
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            return_tensors="pt",
            return_dict=True
        ).to(model.device)
        
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=150,
                do_sample=False,
            )
        
        # 생성된 토큰만 추출
        generated_ids = output[0][inputs['input_ids'].shape[1]:]
        caption = tokenizer.decode(generated_ids, skip_special_tokens=True)
        
        return caption
    except Exception as e:
        print(f"캡셔닝 오류: {str(e)}")
        traceback.print_exc()
        return None
    finally:
        torch.cuda.empty_cache()

# 2. 배치 처리
bucket_name = 'flexmatch-data'
keys = final_img_path_df['s3_path'].to_list()[21103:]  # 전체 데이터
batch_size = 100 
output_file = "qwen_emotion_captions_2.jsonl"

# 기존 결과 확인 (중단됐을 경우 재개)
processed_keys = set()
if os.path.exists(output_file):
    with open(output_file, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            processed_keys.add(data['key'])
    print(f"이전에 처리한 파일: {len(processed_keys)}개 (재개 모드)")
else:
    print("새로 시작합니다")

# 배치 처리
print(f"\n총 {len(keys)}개 이미지 캡셔닝 시작...")
print(f"배치 크기: {batch_size}, 예상 배치 수: {len(keys)//batch_size + 1}")

with open(output_file, 'a', encoding='utf-8') as f:
    for i, key in enumerate(tqdm(keys, desc="이미지 캡셔닝")):
        # 이미 처리한 것은 스킵
        if key in processed_keys:
            continue
        
        # 이미지 로드
        img = load_image_from_s3(bucket_name, key)
        caption = None
        
        if img is not None:
            caption = caption_image(img, model, tokenizer)
            caption = clean_caption(caption)  # 아티팩트 제거
        
        # 결과 저장
        result = {"key": key, "caption": caption}
        f.write(json.dumps(result, ensure_ascii=False) + '\n')
        f.flush()
        
        # 메모리 정리
        if img is not None:
            del img
        gc.collect()
        
        # 배치마다 GPU 캐시 정리
        if (i + 1) % batch_size == 0:
            torch.cuda.empty_cache()
            completed = len(processed_keys) + (i + 1)
            percentage = (completed / len(keys)) * 100
            print(f"\n배치 {(i+1)//batch_size} 완료 ({completed}/{len(keys)}, {percentage:.1f}%), GPU 메모리 정리됨")

print("\n" + "="*50)
print("캡셔닝 완료!")
print("="*50)

# 3. 최종 통계
total_processed = len(processed_keys) + len(keys)
with open(output_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    total_lines = len(lines)
    success_count = sum(1 for line in lines if json.loads(line).get('caption') is not None)

print(f"\n최종 통계:")
print(f"  총 이미지: {len(keys)}개")
print(f"  처리 완료: {total_lines}개")
print(f"  성공: {success_count}개 ({success_count/total_lines*100:.1f}%)")
print(f"  실패: {total_lines - success_count}개")

# 4. 샘플 결과
print("\n=== 샘플 결과 (마지막 3개) ===")
try:
    with open(output_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines[-3:]:
            data = json.loads(line)
            if data['caption']:
                print(f"\n키: {data['key']}")
                print(f"캡션:\n{data['caption']}")
            else:
                print(f"\n키: {data['key']}")
                print(f"캡션: [실패]")
except Exception as e:
    print(f"샘플 읽기 오류: {e}")

# 메모리 정리
del model
torch.cuda.empty_cache()